In [1]:
from dotenv import load_dotenv

load_dotenv('.env')

import os
os.environ['CURL_CA_BUNDLE'] = ''

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Load Models

In [3]:
from langchain import PromptTemplate, LLMChain, HuggingFaceHub

# initialize Hub LLM
llm_t5 = HuggingFaceHub(
    repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0,"max_length": 64,"max_new_tokens":128}
)

llm_mistral = HuggingFaceHub(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    model_kwargs={'temperature':0.5,"max_length": 64,"max_new_tokens":512}
)

C:\Users\Ghost\AppData\Local\Temp\ipykernel_1700\2645606937.py:4: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  llm_t5 = HuggingFaceHub(


# Role Prompting

In [5]:
from langchain.prompts import PromptTemplate

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# Create LLMChain
chain_t5 = LLMChain(llm=llm_t5, prompt=prompt)
chain_mistral = LLMChain(llm=llm_mistral, prompt=prompt)

C:\Users\Ghost\AppData\Local\Temp\ipykernel_7904\3691325039.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain_t5 = LLMChain(llm=llm_t5, prompt=prompt)


In [10]:
# Run the LLMChain to get the AI-generated song title
response = chain_t5.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: Spaced Out


In [11]:
# Run the LLMChain to get the AI-generated song title
response = chain_mistral.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: 
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about interstellar travel in the year 3030?



# Few Shots Prompting

In [4]:
from langchain import FewShotPromptTemplate

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain_t5 = LLMChain(llm=llm_t5, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))
chain_mistral = LLMChain(llm=llm_mistral, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))

C:\Users\Ghost\AppData\Local\Temp\ipykernel_1700\935672472.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain_t5 = LLMChain(llm=llm_t5, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))


In [5]:
# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain_t5.run({})

print("Color: purple")
print("Emotion:", response)

C:\Users\Ghost\AppData\Local\Temp\ipykernel_1700\2768533074.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain_t5.run({})


Color: purple
Emotion: love


In [6]:
# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain_mistral.run({})

print("Color: purple")
print("Emotion:", response)

KeyboardInterrupt: 

# Bad Prompt Practises

In [9]:
from langchain import PromptTemplate

template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="dogs")

'Tell me something about dogs.'

# Chain Prompting

In [10]:
# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm_t5, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm_t5, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: Einstein's theory of general relativity is the basis of the laws of physics.


In [11]:
# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm_mistral, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm_mistral, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: What is the name of the famous scientist who developed the theory of general relativity?
Answer:  Albert Einstein
Fact: Provide a brief description of What is the name of the famous scientist who developed the theory of general relativity?
Answer:  Albert Einstein's theory of general relativity.
Answer:  Albert Einstein is the famous scientist who developed the theory of general relativity. This groundbreaking theory, published in 1915, fundamentally changed our understanding of gravity and the structure of the universe. It replaced the previous Newtonian theory with a new perspective that described gravity as a curvature of spacetime caused by mass and energy. This theory has been confirmed through numerous experiments and observations and remains a cornerstone of modern physics.


# Bad Prompt Example

In [12]:
# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something interesting about {scientist}.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm_t5, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm_t5, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: Einstein was born in 1848


In [13]:
# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something interesting about {scientist}.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm_mistral, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm_mistral, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: What is the name of the famous scientist who developed the theory of general relativity?
Answer:  Albert Einstein
Fact: Tell me something interesting about What is the name of the famous scientist who developed the theory of general relativity?
Answer:  Albert Einstein.
Answer:  That's correct! Albert Einstein's theory of general relativity revolutionized our understanding of gravity and the universe. It describes gravity not as a force, but as a curvature of spacetime caused by mass and energy. This theory has been confirmed through numerous experiments and observations, and continues to shape our scientific understanding today.


# Unclear Prompt

In [14]:
# Prompt 1
template_question = """What are some musical genres?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details.
Answer: """
prompt_fact = PromptTemplate(input_variables=["genre1", "genre2", "genre3"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm_t5, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Assign three hardcoded genres
genre1, genre2, genre3 = "jazz", "pop", "rock"

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm_t5, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3": genre3}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Genres:", genre1, genre2, genre3)
print("Fact:", response_fact)

Genres: jazz pop rock
Fact: jazz is a genre of music that originated in the u.s. , and has been popular in many countries around the world .


In [15]:
# Prompt 1
template_question = """What are some musical genres?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details.
Answer: """
prompt_fact = PromptTemplate(input_variables=["genre1", "genre2", "genre3"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm_mistral, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Assign three hardcoded genres
genre1, genre2, genre3 = "jazz", "pop", "rock"

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm_mistral, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3": genre3}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Genres:", genre1, genre2, genre3)
print("Fact:", response_fact)

Genres: jazz pop rock
Fact: Tell me something about jazz, pop, and rock without giving any specific details.
Answer: 
Jazz is a genre of music characterized by its improvisational nature, complex harmonies, and intricate rhythms. It originated in the late 19th and early 20th centuries, primarily in African American communities in the United States. It's known for its expressive and emotive qualities, often emphasizing individual expression and innovation.
Pop music, on the other hand, is generally more commercial and accessible to a wider audience. It's characterized by its catchy melodies, simple song structures, and danceable rhythms. Pop music has its roots in various genres, including jazz, blues, and rock and roll, and has evolved over the decades to reflect changing cultural trends and technologies.
Rock music is a broad genre that encompasses a wide range of styles and influences. It's characterized by its strong rhythms, powerful instrumentation, and often rebellious or counter

# Chain of Thought Prompting

In [16]:
examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few-shot prompt template
chain = LLMChain(llm=llm_t5, prompt=few_shot_prompt_template)

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the LLMChain for the user query
response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response: Getting to know your audience and learning how to listen.


In [17]:
examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few-shot prompt template
chain = LLMChain(llm=llm_mistral, prompt=few_shot_prompt_template)

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the LLMChain for the user query
response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response: The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 



User: What's the secret to happiness?
AI: Finding balance in life and learning to enjoy the small moments.



User: How can I become more productive?
AI: Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance.



User: What are some tips for improving communication skills?
AI: 1. Active listening, 2. Use clear and concise language, 3. Practice empathy, and 4. Be open to feedback.



User: How do I deal with stress?
AI: 1. Identify the source of stress, 2. Practice relaxation techniques, 3. Get enough sleep, and 4. Stay active.



User: How can I boost my self-confidence?
AI: 1. Identify and challenge negative thoughts, 2. Set achievable goals, 3. Practice self-care, and 4. Surround yourself with positive i